## what is tokenization?

it's the process of breaking text into smaller units (tokens) which are the input to the LLM. these tokens can be characters, subwords, words and byte sequences. the way text is tokenized directly affects model performance, generalization, vocabulary size and handling of rare/unknown words.

we can't use whole words as tokens for a few reasons:

1. the vocabulary quickly becomes very large.
2. models struggle with rare or unseen words, such as typos or new names.
3. morphologically rich languages (like german) create many variants of the same root word.

this is why we implement **subword tokenization**, splitting words into subwords to reduce total vocabulary size but retain the complex relations in the text.

- since the corpus that trains the tokenizer is almost entirely composed of english, the tokenization of other languages usually result in a higher vocab size, because the "chunks" are more broken up and we use a lot more tokens for the *exact same thing*, bloating the sequence length of the documents and overflowing the maximum context length of attention.


## byte-pair encoding
is a popular subowrd tokenization approach. the most common pair of bytes in a corpus are iteratively combined until the necessary vocabulary size is attained, creating a collection of subwords. 

1. start with a base vocabulary of characters (ex. 'a','b','c',...).
2. represent each word as a sequence of characters + end-of-word symbol.
3. count the most frequent pair of symbols in the training data.
4. merge the most frequent pair into a new symbol and update all occurrences.
5. repeat 3-4 for a fixed number of merges or until the vocabulary reaches a certain size.

the main advantages of BPE is how well it adapts to rare words (by breaking them into known subwords) and how it limits vocabulary size. also, it remains robust across many languages.

however, BPE is frequency-based, not linguistically motivated, so some splits may break grammar relations. also, merges are fixed, so the tokenizer doesn't adapt to new domains, and it may treat semantically similar words differently if they tokenize differently.

**example:** considering the sequence ```aaabdaaabac```:

1. the byte pair ``aa`` appears twice, so we make it a new byte pair ``Z``: ```ZabdZabac```
2. the byte pair ``ab`` appears twice, so we make it a new byte pair ``Y``: ``ZYbZYac``
3. the byte pair ``ZY`` appears twice, so we can make a new byte pair ``X``: ``XbXac``
4. there are no more byte pairs that appear more than once, so we're done.

to de-tokenize the data, we simply perform the replacements in the reverse order.


## embedding table
after tokenization, our corpus becomes a sequence of token IDs, like

In [ ]:
"unbelievable" -> ["un","believ","able"] -> [152,6201,398]

but neural networks don't operate on IDs, they need numerical vectors, so each token ID is **mapped to a dense vector** using the embedding table. 

the embedding table is a ```vocab_size X embedding_dim``` matrix where each row corresponds to a token. when the model sees a token ID ```i```, it looks up row i to get the token's embedding. this is the vector that is fed into the transformer. 

it is **learned during training**, captures semantic meaning (similar words or subwords have similar vectors) and it allows the model to generalize across words or subwords with similar usage.



## coding our tokenizer
first, we need to understand that python uses *unicode* for encoding each character. specifically, we usually use UTF-8 to take unicode text and transform it into binary strings. 

In [1]:
print(ord("h"), ord("👋"), ord("愛"))

104 128075 24859


In [ ]:
list(("こんにちは 👋 olá!").encode("utf-8"))

if we just used UTF-8 as our tokens, we would have a vocabulary length of 256 possible tokens, which is very small, and would make our text streched out over very long sequences of bytes, messing up our context length. 

since we don't want to use our raw bytes, we turn to the BPE algorithm to compress the byte sequences. for this example, i chose the first paragraph of tolstoy's anna karenina.

In [12]:
text = "Happy families are all alike; every unhappy family is unhappy in its own way. Everything was in confusion in the Oblonskys' house. The wife had discovered that the husband was carrying on an intrigue with a French girl, who had been a governess in their family, and she had announced to her husband that she could not go on living in the same house with him. This position of affairs had now lasted three days, and not only the husband and wife themselves, but all the members of their family and household, were painfully conscious of it. Every person in the house felt that there was so sense in their living together, and that the stray people brought together by chance in any inn had more in common with one another than they, the members of the family and household of the Oblonskys. The wife did not leave her own room, the husband had not been at home for three days. The children ran wild all over the house; the English governess quarreled with the housekeeper, and wrote to a friend asking her to look out for a new situation for her; the man-cook had walked off the day before just at dinner time; the kitchen-maid, and the coachman had given warning."
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))

print(f"text length: {len(text)}\ntoken length: {len(tokens)}")

text length: 1163
token length: 1163


In [20]:
def find_pairs(ids):
    '''
    parameters:
    - ids: a list of integers.
    returns:
    - counts: a dictionary whose key is the pair and value is the amount of time they appeared.
    iterates on every consecutive element in the ids vector and adds one to counts for every pair found
    '''
    counts = {}
    for pair in zip(ids, ids[1:]): # iterating consecutive elements
        counts[pair] = counts.get(pair,0) + 1 # incrementing for each pair found
    return counts

now, we can see how often each pair appears when applying the tokenized text into this function. the pair most often found was ```(101, 32)```, which represents the characters ```'e'``` and ```' '```.

In [ ]:
stats = find_pairs(tokens)
print(sorted(((v,k) for k,v in stats.items()), reverse = True)) # v = # found, k = pair

we have the pairs, so we'll iterate over this entire list and start minting the paired bytes, starting for the most common. since we're working with UTF-8 encoding, which has a maximum value of 255, we will start our new tokens at 256, and grow from there. so the new token for the pair ```(101,32)``` will be ```(256)```.

In [28]:
def merge(ids, pair, idx):
    '''
    parameters:
    - ids: a list of integers
    - pair: the pair we want to replace
    - idx: the token we will replace them with
    returns:
    - new_ids: a list of integers with every instance of pair changed for idx
    replaces every instance of the pair with idx
    '''
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids

In [29]:
top_pair = max(stats, key=stats.get)

tokens2 = merge(tokens, top_pair, 256)
print("length: ", len(tokens2))

length:  1119


which makes total sense, because our original tokenized length was 1163 and we had 44 occurences of the pair ```(101, 32)```, so now our length is 1163 - 44 = 1119.

now we just iterate this function. how much we iterate is a *hyperparameter*: the more steps we take, the larger will be our vocabulary and the shorter will be our sequences. we need to find the sweet spot that works the best in practice.

- for example, GPT-4 uses around 50.000 tokens.

In [ ]:
vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens) # copy so we don't destroy the original list

merges = {}
for i in range(num_merges):
    stats = find_pairs(ids)
    top_pair = max(stats,key=stats.get)
    idx = 256 + i
    print(f"Merging {top_pair} into {idx}")
    ids = merge(ids, top_pair, idx)
    merges[top_pair] = idx

In [41]:
print(f"tokens length: {len(tokens)}\ncompressed length: {len(ids)}\ncompression ratio: {len(tokens) / len(ids):.2f}x")

tokens length: 1163
compressed length: 821
compression ratio: 1.42x


the choice of the number of merges, as discussed, is a hyperparameter. in this example i chose to do exactly 20 merges, which was enough to compress the text by 1.42x!

the merges dictionary works as an inverse tree, where we start with the leaves and build up the new tokens from them. it's necessary for us to see what changes the tokenizer did. 

## but where actually is the tokenizer?
the tokenizer is a **completely separate, independent module** from the LLM. it has it's own training set of text (which is can be different from that of the LLM). it then translates back and forth between raw text and sequences of tokens. the LLM only ever sees the tokens and *never* directly deals with any text.

once our tokenizer is trained (it has both the vocabulary and the merges), we can do the encoding and decoding. 

## decoding
given a sequence of integers in the range ```[ 0, vocab_size]```, how can we get a string object?

1. first up, we build a ```vocab``` dictionary populated by all possible single-byte values, each mapped to its byte representation.
2. then we add the merges we did, available on the merge dictionary. for example, if we merged the pair ``'a'`` and ``'b'``, we got ``'ab'``, it will be represented in ``vocab``.
3. to find the real text, we concatenate all the token byte sequence together into one byte object and decode it.

In [46]:
vocab = {idx: bytes([idx]) for idx in range (256)}
for (pair0, pair1), idx in merges.items():
    vocab[idx] = vocab[pair0] + vocab[pair1]

def decode(ids):
    '''
    parameters:
    - ids: the tokens of our texto
    returns:
    - text: the string decoded
    '''
    tokens = b"".join(vocab[idx] for idx in ids) # one way of concatenating bytes together
    text = tokens.decode("utf-8", errors="replace")
    return text